## 1. Import

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings(action='ignore')

## 2. Data Load

In [26]:
# 학습/평가 데이터 로드
train_df = pd.read_csv('C:/Project/DACON_DEFAULT_PREDICTION/data/default_prediction/train.csv').drop(columns=['UID'])
test_df = pd.read_csv('C:/Project/DACON_DEFAULT_PREDICTION/data/default_prediction/test.csv').drop(columns=['UID'])

### 2-1. 데이터 조회

In [27]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   주거 형태              10000 non-null  object 
 1   연간 소득              10000 non-null  float64
 2   현재 직장 근속 연수        10000 non-null  object 
 3   체납 세금 압류 횟수        10000 non-null  float64
 4   개설된 신용계좌 수         10000 non-null  int64  
 5   신용 거래 연수           10000 non-null  float64
 6   최대 신용한도            10000 non-null  float64
 7   신용 문제 발생 횟수        10000 non-null  int64  
 8   마지막 연체 이후 경과 개월 수  10000 non-null  int64  
 9   개인 파산 횟수           10000 non-null  int64  
 10  대출 목적              10000 non-null  object 
 11  대출 상환 기간           10000 non-null  object 
 12  현재 대출 잔액           10000 non-null  float64
 13  현재 미상환 신용액         10000 non-null  float64
 14  월 상환 부채액           10000 non-null  float64
 15  신용 점수              10000 non-null  int64  
 16  채무 불이행 여부          1000

In [28]:
train_df.describe()

,연간 소득,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
count,1.000000e+04,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.00000,10000.000000,10000.000000,1.000000e+04,1.000000e+04,10000.00000,10000.000000,10000.000000
mean,2.163959e+06,0.199300,12.248900,19.879360,1.175265e+06,0.62620,30.889200,0.373200,5.061200e+05,3.649126e+05,22367.28075,744.215000,0.341200
std,1.434430e+06,0.714304,4.620572,7.206693,1.604199e+06,1.23419,20.011561,0.843797,2.831462e+05,3.537942e+05,15186.49738,56.995698,0.474136
min,2.676210e+05,0.000000,2.000000,6.000000,0.000000e+00,0.00000,0.000000,0.000000,2.917650e+04,0.000000e+00,0.00000,502.000000,0.000000
25%,1.311437e+06,0.000000,9.000000,14.600000,4.482476e+05,0.00000,14.000000,0.000000,3.001901e+05,1.471856e+05,10893.75000,704.000000,0.000000
50%,1.743223e+06,0.000000,12.000000,17.950000,7.670910e+05,0.00000,28.000000,0.000000,4.743412e+05,2.547930e+05,20160.00000,756.000000,0.000000
75%,2.447664e+06,0.000000,15.000000,24.100000,1.147283e+06,1.00000,41.000000,0.000000,5.919079e+05,4.749180e+05,30647.25000,793.000000,1.000000
max,1.722975e+07,7.000000,35.000000,51.700000,2.323233e+07,7.00000,88.000000,3.000000,1.373613e+06,3.946300e+06,153574.50000,825.000000,1.000000


In [29]:
h_t = pd.concat([train_df.head(), train_df.tail()]).reset_index(drop=True)
h_t

,주거 형태,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,대출 목적,대출 상환 기간,현재 대출 잔액,현재 미상환 신용액,월 상환 부채액,신용 점수,채무 불이행 여부
0,자가,1941337.5,10년 이상,0.0,9,13.4,400597.5,0,24,1,부채 통합,단기 상환,390903.0,225457.5,8806.5,767,0
1,월세,1979505.0,10년 이상,0.0,5,15.1,360679.5,0,11,0,부채 통합,단기 상환,1002184.5,64749.0,24961.5,767,0
2,월세,1356381.0,4년,0.0,12,18.8,491770.5,1,74,3,부채 통합,단기 상환,227775.0,487644.0,12069.0,800,1
3,월세,1049017.5,6년,0.0,15,14.8,411546.0,1,22,1,부채 통합,단기 상환,251383.5,413211.0,31749.0,796,1
4,월세,4320217.5,2년,0.0,11,26.1,895288.5,0,32,0,부채 통합,장기 상환,1163176.5,78991.5,5862.0,751,0
5,주택 담보 대출 (거주 중),1339473.0,10년 이상,0.0,9,18.7,319027.5,0,68,0,부채 통합,단기 상환,126216.0,177028.5,6237.0,755,0
6,주택 담보 대출 (거주 중),2297230.5,2년,0.0,11,28.3,399799.5,0,7,0,주택 개보수,장기 상환,371907.0,347449.5,53301.0,707,0
7,주택 담보 대출 (거주 중),1221523.5,10년 이상,0.0,9,30.1,823305.0,0,14,0,부채 통합,장기 상환,869736.0,176905.5,11436.0,733,0
8,자가,3343584.0,10년 이상,0.0,10,20.3,724314.0,0,25,0,부채 통합,단기 상환,443008.5,139294.5,25567.5,696,0
9,주택 담보 대출 (거주 중),2175133.5,5년,0.0,5,24.9,52999.5,0,9,0,주택 개보수,장기 상환,1152918.0,106930.5,12676.5,676,0


In [30]:
abs(train_df.corr(numeric_only=True)['채무 불이행 여부']).drop('채무 불이행 여부').sort_values()

현재 미상환 신용액           0.000048
마지막 연체 이후 경과 개월 수    0.012058
월 상환 부채액             0.020135
체납 세금 압류 횟수          0.030419
현재 대출 잔액             0.033052
신용 문제 발생 횟수          0.044847
최대 신용한도              0.044997
신용 점수                0.095783
신용 거래 연수             0.108856
개인 파산 횟수             0.115900
연간 소득                0.120499
개설된 신용계좌 수           0.169900
Name: 채무 불이행 여부, dtype: float64

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplot

corr = train_df.corr(numeric_only=True)
plt.figure(figsize=[10,8])
sns.heatmap(corr, annot=True).round(2)
plt.show()

ModuleNotFoundError: No module named 'koreanize_matplot'

## 3. Pre-processing (전처리)

In [ ]:
categorical_col = [
    '주거 형태',
    '현재 직장 근속 연수',
    '대출 목적',
    '대출 상환 기간'
]

# OneHotEncoder 초기화
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# 훈련 데이터에 대해 인코더 학습
encoder.fit(train_df[categorical_col])

# 훈련 데이터와 테스트 데이터 변환
train_encoded = encoder.transform(train_df[categorical_col])
test_encoded = encoder.transform(test_df[categorical_col])

# One-hot encoding 결과를 데이터프레임으로 변환
train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out(categorical_col))
test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out(categorical_col))

# 인코딩된 결과를 원래 데이터에 적용
train_df = pd.concat([train_df.drop(columns=categorical_col).reset_index(drop=True), train_encoded_df], axis=1)
test_df = pd.concat([test_df.drop(columns=categorical_col).reset_index(drop=True), test_encoded_df], axis=1)

## 4. Train / Validation Split (학습 데이터 분할)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    train_df.drop(columns=['채무 불이행 여부']), 
    train_df['채무 불이행 여부'], 
    test_size=0.2, 
    random_state=42
)

## 5. Model Training (모델 학습)

In [ ]:
# XGBoost 모델 학습
model = XGBClassifier(
    n_estimators=100,  # 트리 개수
    max_depth=5,       # 최대 깊이
    learning_rate=0.15, # 학습률
    random_state=42,
    eval_metric="auc",  # 평가 지표 설정
)

# 학습 및 Validation 성능 모니터링
eval_set = [(X_train, y_train), (X_val, y_val)]
model.fit(
    X_train, y_train,
    eval_set=eval_set,
    verbose=True
    # early_stopping_rounds=10
)


[0]	validation_0-auc:0.73056	validation_1-auc:0.67897
[1]	validation_0-auc:0.74434	validation_1-auc:0.69985
[2]	validation_0-auc:0.75222	validation_1-auc:0.70013
[3]	validation_0-auc:0.75962	validation_1-auc:0.70342
[4]	validation_0-auc:0.76341	validation_1-auc:0.70646
[5]	validation_0-auc:0.76785	validation_1-auc:0.70622
[6]	validation_0-auc:0.77482	validation_1-auc:0.71070
[7]	validation_0-auc:0.78105	validation_1-auc:0.71547
[8]	validation_0-auc:0.78692	validation_1-auc:0.71940
[9]	validation_0-auc:0.78952	validation_1-auc:0.72130
[10]	validation_0-auc:0.79418	validation_1-auc:0.72203
[11]	validation_0-auc:0.79838	validation_1-auc:0.72385
[12]	validation_0-auc:0.80065	validation_1-auc:0.72424
[13]	validation_0-auc:0.80335	validation_1-auc:0.72626
[14]	validation_0-auc:0.80676	validation_1-auc:0.72740
[15]	validation_0-auc:0.80895	validation_1-auc:0.72852
[16]	validation_0-auc:0.81204	validation_1-auc:0.72941
[17]	validation_0-auc:0.81473	validation_1-auc:0.72946
[18]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

## 6. Prediction

In [ ]:
# 채무 불이행 '확률'을 예측합니다.
preds = model.predict_proba(test_df)[:,1]

## 7. Submission (제출 파일 생성)

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

# 결과 저장
submit['채무 불이행 확률'] = preds
submit.to_csv('./submission.csv', encoding='UTF-8-sig', index=False)